In [1]:
import tensorflow as tf
from tqdm import tqdm
tf.keras.backend.set_floatx('float32')

In [2]:
from data_management.VOC2012ManagerObjDetection import VOC2012ManagerObjDetection
db_manager = VOC2012ManagerObjDetection("C:/Users/antho/Documents/GitHub/data/VOCdevkit/VOC2012/",
                                        batch_size=32)

In [3]:
from models.SSD300 import SSD300
SSD300_model = SSD300(21)
SSD300_model.build(input_shape=(32, 300, 300, 3))
SSD300_model.load_vgg16_imagenet_weights()

In [4]:
optimizer = tf.keras.optimizers.SGD(
    learning_rate=10**-3,
    momentum=0.9)

In [9]:
import numpy as np

In [12]:
for epoch in range(250):
    losses = []
    for batch in tqdm(db_manager.batches):
        # get data from batch
        images, confs_gt, locs_gt = db_manager.getImagesAndGtSpeedUp(batch, SSD300_model.default_boxes)

        # get predictions and losses
        with tf.GradientTape() as tape:
            confs_pred, locs_pred = SSD300_model(images)
            # concat tensor to delete the dimension of feature maps
            confs_pred = tf.concat(confs_pred, axis=1)
            locs_pred = tf.concat(locs_pred, axis=1)

            # calculate loss
            confs_loss, locs_loss = SSD300_model.calculateLoss(
                confs_pred, confs_gt, locs_pred, locs_gt)
            loss = confs_loss + 1*locs_loss
            losses.append(loss)

        # back propagation
        gradients = tape.gradient(loss, SSD300_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, SSD300_model.trainable_variables))
    if epoch % 1 == 0:
        print("Loss epoch {} = {}".format(epoch, np.mean(losses, axis=0)))
    if epoch % 1 == 0:
        SSD300_model.save_weights("ssd_weights_epoch_{:03d}.h5".format(epoch+4))

  0%|                                                                                          | 0/374 [00:00<?, ?it/s]

Loss epoch 0 = 4.37109375


  0%|                                                                                          | 0/374 [00:00<?, ?it/s]

Loss epoch 1 = 4.34375


  0%|                                                                                          | 0/374 [00:00<?, ?it/s]

Loss epoch 2 = 4.33984375


  0%|▏                                                                                 | 1/374 [00:08<51:15,  8.25s/it]


KeyboardInterrupt: 